In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
# from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler
import xgboost
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier

In [19]:
path = r'C:\Users\Public\Documents\HRS\28.11'
beginning_of_name_EL = '\Feature matrix EL no noise'
beginning_of_name_EL_OM = '\Feature matrix EL+OM no noise'
end_of_name = 'peaks.csv'
list_of_names = [' 3 ', ' 4 ', ' 5 ', ' no 72_F75 3 ', ' no 72_F75 4 ',  ' no 72_F75 5 ']

acc = np.zeros((6,3)) # list of accuracies, shape list_of_names*3
f1 = np.zeros((6,3)) # list of f1_scores, shape list_of_names*3


data1 = pd.read_csv(path + beginning_of_name_EL_OM + list_of_names[1] + end_of_name, index_col=0)
target=data1['target']
data2 = data1.iloc[:,:int((data1.shape[1]-1)/2)]
data2 = pd.concat([data2, target], axis=1)
# print(data2.head())
# print(y1)
# (data1.shape[1]-1)/2

   #peaks_0.05  #peaks_0.1     freq 0     freq 1     freq 2     freq 3  \
0           47          37  16.333333  14.583333  16.500000  14.083333   
1           46          30  12.833333  12.000000  11.083333  12.666667   
2           39          26  15.500000  17.000000  14.583333  16.500000   
3           46          26  17.000000  16.166667  15.500000  16.500000   
4            4           2   8.487334  16.851663   1.722068   1.353053   

   height 0  height 1  height 2  height 3   width 0   width 1   width 2  \
0  0.009280  0.008940  0.008834  0.008347  5.326312  1.064753  0.981352   
1  0.014327  0.012430  0.011112  0.010166  1.334530  2.196597  1.186796   
2  0.009945  0.009912  0.009245  0.009045  2.896952  1.228006  1.379949   
3  0.020191  0.017435  0.017393  0.015742  2.095364  1.793380  1.461269   
4  0.013458  0.008602  0.004335  0.003909  2.110823  1.347647  6.265876   

    width 3  prominence 0  prominence 1  prominence 2  prominence 3  \
0  1.229243      0.008569      0.

In [20]:
path = r'C:\Users\Public\Documents\HRS\28.11'
beginning_of_name_EL = '\Feature matrix EL no noise'
beginning_of_name_EL_OM = '\Feature matrix EL+OM no noise'
end_of_name = 'peaks.csv'
list_of_names = [' 3 ', ' 4 ', ' 5 ', ' no 72_F75 3 ', ' no 72_F75 4 ',  ' no 72_F75 5 ']

acc = np.zeros((6,3)) # list of accuracies, shape list_of_names*3
f1 = np.zeros((6,3)) # list of f1_scores, shape list_of_names*3
for i in range(len(list_of_names)):
    data = pd.read_csv(path + beginning_of_name_EL + list_of_names[i] + end_of_name, index_col=0)
    data1 = pd.read_csv(path + beginning_of_name_EL_OM + list_of_names[i] + end_of_name, index_col=0)
    data.drop(['Label for OM and EL'], axis=1, inplace=True)
    print(i)
    
    X = data.drop('target', 1) #full of EL data ~ 900 sequencies
    y = data['target']

    X1 = data1.drop('target', 1) #full of OM data ~ 200 sequencies
    y1 = data1['target']
    
    target=data1['target']
    data2 = data1.iloc[:,:int((data1.shape[1]-1)/2)]
    data2 = pd.concat([data2, target], axis=1)
    
    X2 = data2.drop('target', 1) #cutted EL data to the size of OM data ~ 200 sequencies
    y2 = data2['target']

    (X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size=0.3, stratify=y)
    encoder = StandardScaler()
    X_train_scaled = encoder.fit_transform(X_train)
    X_test_scaled = encoder.fit_transform(X_test)
    X_resampled, y_resampled = SMOTE().fit_resample(X_train_scaled, y_train)

    (X_train_1, X_test_1, y_train_1, y_test_1) = train_test_split(X1, y1, test_size=0.3, stratify=y1)
    encoder = StandardScaler()
    X_train_scaled_1 = encoder.fit_transform(X_train_1)
    X_test_scaled_1 = encoder.fit_transform(X_test_1)
    X_resampled_1, y_resampled_1 = SMOTE().fit_resample(X_train_scaled_1, y_train_1)

    (X_train_2, X_test_2, y_train_2, y_test_2) = train_test_split(X2, y2, test_size=0.3, stratify=y2)
    encoder = StandardScaler()
    X_train_scaled_2 = encoder.fit_transform(X_train_2)
    X_test_scaled_2 = encoder.fit_transform(X_test_2)
    X_resampled_2, y_resampled_2 = SMOTE().fit_resample(X_train_scaled_2, y_train_2)
    
    model = RandomForestClassifier(n_estimators = 1000, n_jobs = -1)
    model.fit(X_resampled, y_resampled)

    model1 = RandomForestClassifier(n_estimators = 1000, n_jobs = -1)
    model1.fit(X_resampled_1, y_resampled_1)

    model2 = RandomForestClassifier(n_estimators = 1000, n_jobs = -1)
    model2.fit(X_resampled_2, y_resampled_2)

    y_pred = model.predict(X_test_scaled)
#     predictions = [round(value) for value in y_pred]

    y_pred1 = model1.predict(X_test_scaled_1)
#     predictions1 = [round(value) for value in y_pred1]

    y_pred2 = model2.predict(X_test_scaled_2)
#     predictions2 = [round(value) for value in y_pred2]
    
    acc[i][0]= (accuracy_score(y_test, y_pred) * 100.0)
    acc[i][1]= (accuracy_score(y_test_1, y_pred1) * 100.0)
    acc[i][2]= (accuracy_score(y_test_2, y_pred2) * 100.0)
    
    f1[i][0]= f1_score(y_test, y_pred)
    f1[i][1]= f1_score(y_test_1, y_pred1)
    f1[i][2]= f1_score(y_test_2, y_pred2)

0


C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype in

1


C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype in

2


...

C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype in

4


C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype in

5


C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype in

In [21]:
acc

array([[86.11111111, 74.35897436, 75.64102564],
       [89.28571429, 67.94871795, 70.51282051],
       [86.11111111, 79.48717949, 75.64102564],
       [87.60683761, 72.60273973, 79.45205479],
       [88.46153846, 73.97260274, 78.08219178],
       [88.88888889, 78.08219178, 80.82191781]])

In [22]:
f1

array([[0.14634146, 0.23076923, 0.24      ],
       [0.22857143, 0.24242424, 0.20689655],
       [0.05405405, 0.42857143, 0.17391304],
       [0.        , 0.        , 0.28571429],
       [0.12903226, 0.0952381 , 0.2       ],
       [0.27777778, 0.        , 0.125     ]])

In [23]:
path = r'C:\Users\Public\Documents\HRS\28.11'
beginning_of_name_EL = '\Feature matrix EL no noise'
beginning_of_name_EL_OM = '\Feature matrix EL+OM no noise'
end_of_name = 'peaks.csv'
list_of_names = [' 3 ', ' 4 ', ' 5 ', ' no 72_F75 3 ', ' no 72_F75 4 ',  ' no 72_F75 5 ']

acc = np.zeros((6,3)) # list of accuracies, shape list_of_names*3
f1 = np.zeros((6,3)) # list of f1_scores, shape list_of_names*3
for i in range(len(list_of_names)):
    data = pd.read_csv(path + beginning_of_name_EL + list_of_names[i] + end_of_name, index_col=0)
    data1 = pd.read_csv(path + beginning_of_name_EL_OM + list_of_names[i] + end_of_name, index_col=0)
    data.drop(['Label for OM and EL'], axis=1, inplace=True)
    print(i)
    
    X = data.drop('target', 1) #full of EL data ~ 900 sequencies
    y = data['target']

    X1 = data1.drop('target', 1) #full of OM data ~ 200 sequencies
    y1 = data1['target']
    
    target=data1['target']
    data2 = data1.iloc[:,:int((data1.shape[1]-1)/2)]
    data2 = pd.concat([data2, target], axis=1)
    
    X2 = data2.drop('target', 1) #cutted EL data to the size of OM data ~ 200 sequencies
    y2 = data2['target']

    scaler = StandardScaler()
    
    X = scaler.fit_transform(X)
    X1 = scaler.fit_transform(X1)
    X2 = scaler.fit_transform(X2)
    
    X_resampled, y_resampled = SMOTE().fit_resample(X, y)
#     print(X_resampled.shape, y_resampled.shape)
    X1_resampled, y1_resampled = SMOTE().fit_resample(X1, y1)
#     print(X1_resampled.shape, y1_resampled.shape)
    X2_resampled, y2_resampled = SMOTE().fit_resample(X2, y2)
    
    (X_train, X_test, y_train, y_test) = train_test_split(X_resampled, y_resampled, test_size=0.3, stratify=y_resampled)

    (X_train_1, X_test_1, y_train_1, y_test_1) = train_test_split(X1_resampled, y1_resampled, test_size=0.3, stratify=y1_resampled)

    (X_train_2, X_test_2, y_train_2, y_test_2) = train_test_split(X2_resampled, y2_resampled, test_size=0.3, stratify=y2_resampled)
    
    model = RandomForestClassifier(n_estimators = 1000, n_jobs = -1)
    model.fit(X_train, y_train)

    model1 = RandomForestClassifier(n_estimators = 1000, n_jobs = -1)
    model1.fit(X_train_1, y_train_1)

    model2 = RandomForestClassifier(n_estimators = 1000, n_jobs = -1)
    model2.fit(X_train_2, y_train_2)

    y_pred = model.predict(X_test)
#     predictions = [round(value) for value in y_pred]

    y_pred1 = model1.predict(X_test_1)
#     predictions1 = [round(value) for value in y_pred1]

    y_pred2 = model2.predict(X_test_2)
#     predictions2 = [round(value) for value in y_pred2]
    
    acc[i][0]= (accuracy_score(y_test, y_pred) * 100.0)
    acc[i][1]= (accuracy_score(y_test_1, y_pred1) * 100.0)
    acc[i][2]= (accuracy_score(y_test_2, y_pred2) * 100.0)
    
    f1[i][0]= f1_score(y_test, y_pred)
    f1[i][1]= f1_score(y_test_1, y_pred1)
    f1[i][2]= f1_score(y_test_2, y_pred2)

0


C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype in

1


C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype in

2


...

C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype in

4


C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype in

5


C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype in

In [24]:
f1

array([[0.94583333, 0.86821705, 0.8358209 ],
       [0.95597484, 0.89922481, 0.84615385],
       [0.95762712, 0.9375    , 0.85496183],
       [0.93693694, 0.91891892, 0.85714286],
       [0.95535714, 0.87096774, 0.87603306],
       [0.95749441, 0.91056911, 0.9       ]])

In [25]:
acc

array([[94.47983015, 86.61417323, 82.67716535],
       [95.54140127, 89.76377953, 84.2519685 ],
       [95.7537155 , 93.7007874 , 85.03937008],
       [93.59267735, 92.37288136, 85.59322034],
       [95.42334096, 86.44067797, 87.28813559],
       [95.65217391, 90.6779661 , 89.83050847]])